In [ ]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.preprocessing import StandardScaler
from joblib import dump

# Set up for reproducible results
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
# Paths to the .npy files
categories_path = '/Users/rishabhtiwari/Desktop/training_dataset/categories.npy'
valid_periods_path = '/Users/rishabhtiwari/Desktop/training_dataset/valid_periods.npy'
training_data_path = '/Users/rishabhtiwari/Desktop/training_dataset/training_data.npy'

# Load the data
categories = np.load(categories_path)
valid_periods = np.load(valid_periods_path)
training_data = np.load(training_data_path)

In [ ]:
# Constants for sequence and forecast lengths
seq_length = 128
forecast_length = 9

# Initialize two scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [ ]:
# Function to preprocess data
def preprocess_data(data, valid_periods, seq_length, forecast_length, scaler_X, scaler_y):
    X, y = [], []
    for i, row in enumerate(data):
        valid_data = row[valid_periods[i][0]:valid_periods[i][1]]
        if valid_data.size > 0:
            for j in range(len(valid_data) - seq_length - forecast_length + 1):
                seq_X = valid_data[j:(j + seq_length)]
                seq_y = valid_data[(j + seq_length):(j + seq_length + forecast_length)]
                
                X.append(seq_X)
                y.append(seq_y)
    
    X = scaler_X.fit_transform(np.array(X))
    y = scaler_y.fit_transform(np.array(y))

    return np.array(X), np.array(y)

In [ ]:
# Preprocess the data
X, y = preprocess_data(training_data, valid_periods, seq_length, forecast_length, scaler_X, scaler_y)

In [ ]:
# Save the fitted scalers
scaler_X_filename = "/mnt/data/scaler_X.save"
scaler_y_filename = "/mnt/data/scaler_y.save"
dump(scaler_X, scaler_X_filename)
dump(scaler_y, scaler_y_filename)

In [ ]:
# Split the data into training and validation sets
val_size = int(len(X) * 0.2)
indices = np.arange(len(X))
np.random.shuffle(indices)
X_train, y_train = X[:-val_size], y[:-val_size]
X_val, y_val = X[-val_size:], y[-val_size:]

In [ ]:
# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], seq_length, 1))
y_train = y_train.reshape((y_train.shape[0], forecast_length, 1))
X_val = X_val.reshape((X_val.shape[0], seq_length, 1))
y_val = y_val.reshape((y_val.shape[0], forecast_length, 1))

In [ ]:
# Model building function
def build_model(input_shape, lstm_units, dropout_rate, forecast_length):
    input_layer = tfkl.Input(shape=input_shape)
    x = tfkl.Bidirectional(tfkl.LSTM(lstm_units, return_sequences=True))(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    x = tfkl.LSTM(lstm_units // 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    output_layer = tfkl.Dense(forecast_length, activation='linear')(x)  # Linear activation for regression

    model = tfk.Model(inputs=input_layer, outputs=output_layer)
    return model

# Model configuration
input_shape = (seq_length, 1)
dropout_rate = 0.2
lstm_units = 128

In [ ]:
# Build the model
model = build_model(input_shape, lstm_units, dropout_rate, forecast_length)

In [ ]:
# Compile the model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
# Train the model
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Evaluate the model on the validation set (or test set if available)
evaluation_results = model.evaluate(X_val, y_val)
print(f'Validation Loss: {evaluation_results}')

# Save the model
model_save_path = '/mnt/data/my_lstm_model'
model.save(model_save_path)